In [1]:
# set up working catalog
import sys
from pathlib import Path
project_path = str(Path().cwd().parent.parent.resolve())
if project_path not in sys.path:
    sys.path.append(project_path)
    
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
    
# imports 
from common.utils import get_data, get_preprocessor
from common.custom_logistic_regression import CustomLogisticRegressionMulticlass

import pandas as pd
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
data = get_data()
X = data.drop(columns=["Target"])
y = data["Target"]

numerical_column_names = X.select_dtypes(include=["number"]).columns.tolist()
categorical_column_names = X.select_dtypes(include=["object"]).columns.tolist()

preprocessor = get_preprocessor(numerical_column_names, categorical_column_names)

kfold = KFold(n_splits=3, shuffle=True, random_state=16)

data.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,single,2nd phase - general contingent,6th choice,Animation and Multimedia Design,Daytime,Secondary education,122.0,Portuguese,Basic Ed 3rd Cycle,Other - 11th Year,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,single,International student (bachelor),2nd choice,Tourism,Daytime,Secondary education,160.0,Portuguese,Secondary Education - 12th Year or Eq.,Higher Ed - Degree,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,single,1st phase - general contingent,6th choice,Communication Design,Daytime,Secondary education,122.0,Portuguese,Basic Ed 1st Cycle (4th/5th),Basic Ed 1st Cycle (4th/5th),...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,single,2nd phase - general contingent,3rd choice,Journalism and Communication,Daytime,Secondary education,122.0,Portuguese,Basic Ed 2nd Cycle (6th–8th),Basic Ed 1st Cycle (4th/5th),...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,married,Over 23 years old,2nd choice,Social Service (evening attendance),Evening,Secondary education,100.0,Portuguese,Basic Ed 1st Cycle (4th/5th),Basic Ed 2nd Cycle (6th–8th),...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [3]:
def calcualte_using_kfold(pipeline, kfold, X, y):
    results = []
    
    for train_indices, test_indices in kfold.split(X, y):
        X_train = X.iloc[train_indices]
        y_train = y.iloc[train_indices]
        
        X_val = X.iloc[test_indices]
        y_val = y.iloc[test_indices]
        
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_val)
        
        results.append([
            accuracy_score(y_val, y_pred),
            precision_score(y_val, y_pred, average="weighted"),
            recall_score(y_val, y_pred, average="weighted"),
        ])
        
    return results

In [4]:
pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("classifier", LogisticRegression(max_iter=200))
])

close_form_rows = calcualte_using_kfold(pipeline, kfold, X, y)
pd.DataFrame(close_form_rows, columns=["Accuracy", "Precision", "Recall"]).T

,0,1,2
Accuracy,0.765424,0.774915,0.778155
Precision,0.753941,0.761702,0.760945
Recall,0.765424,0.774915,0.778155


In [5]:
pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("classifier", CustomLogisticRegressionMulticlass(epochs=600, batch_size=256, learning_rate=0.01))
])

close_form_rows = calcualte_using_kfold(pipeline, kfold, X, y)
pd.DataFrame(close_form_rows, columns=["Accuracy", "Precision", "Recall"]).T

,0,1,2
Accuracy,0.766102,0.754576,0.774763
Precision,0.746075,0.732913,0.755835
Recall,0.766102,0.754576,0.774763
